# Profiling image loading

In [1]:
import pandas as pd
images = pd.read_csv('data/train.csv')
images.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


## Reading time from disk

In [2]:
from PIL import Image
import time
start = time.time()
for image in images['image_id']:
    im = Image.open('data/train_images/'+image)
    im.load()
end = time.time()
print('duration:', end-start,'s')

duration: 170.23132586479187 s


## Full image processing time

In [3]:
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class ImageDataset(Dataset):
    def __init__(self, images, root_dir, transform=None):
        self.images = images
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.root_dir+'/'+self.images.iloc[idx, 0]
        
        with Image.open(img_name) as image:
            if self.transform:
                image = self.transform(image)

        return (image, self.images.iloc[idx, 1])

In [4]:
image_folder = ImageDataset(images, 
                            root_dir='data/train_images', 
                           transform=transforms.Compose([transforms.Resize((224,224)),                                                         
                                                         transforms.ToTensor(),
                                                         transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                                                        ]))

In [5]:
import torch
start = time.time()
loader = torch.utils.data.DataLoader(image_folder)
for batch in loader:
    _, _ = batch
end = time.time()
print('duration:', end-start,'s')

duration: 309.4250485897064 s


## Switching to albumations

In [6]:
import cv2
class ImageDataset(Dataset):
    def __init__(self, images, root_dir, transform=None):
        self.images = images
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.root_dir+'/'+self.images.iloc[idx, 0]
        
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            
        return (image, self.images.iloc[idx, 1])

In [7]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
image_folder = ImageDataset(images, 
                            root_dir='data/train_images', 
                           transform=A.Compose([A.Resize(224,224),
                                                A.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
                                                ToTensorV2()
                                                ]))

In [8]:
import torch
start = time.time()
loader = torch.utils.data.DataLoader(image_folder)
for batch in loader:
    _, _ = batch
end = time.time()
print('duration:', end-start,'s')

duration: 227.74477171897888 s


30% faster

## Converting to uncompressed image?

In [10]:
import os
if not os.path.exists('data/train_images/png'):
    os.makedirs('data/train_images/png')
    for image in images['image_id']:
        im = Image.open('data/train_images/'+image)
        im.save('data/train_images/png/'+image.replace('jpg','png'), lossless = True)

In [11]:
images_png = images.copy()

In [12]:
images_png['image_id'] = images['image_id'].str.replace('jpg','png')

In [13]:
image_folder = ImageDataset(images_png, 
                            root_dir='data/train_images/png', 
                           transform=A.Compose([A.Resize(224,224),
                                                A.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
                                                ToTensorV2()
                                                ]))

In [14]:
import torch
start = time.time()
loader = torch.utils.data.DataLoader(image_folder)
for batch in loader:
    _, _ = batch
end = time.time()
print('duration:', end-start,'s')

duration: 558.1080112457275 s


Slows down, apperently smaller filesize beats jpegcompression.

## Resize

In [19]:
import os
if not os.path.exists('data/train_images/resize/400'):
    os.makedirs('data/train_images/resize/400')
    for image in images['image_id']:
        im = Image.open('data/train_images/'+image)
        im = im.resize((400,300))
        im.save('data/train_images/resize/400/'+image)

In [22]:
image_folder = ImageDataset(images, 
                            root_dir='data/train_images/resize/400', 
                           transform=A.Compose([A.Resize(224,224),
                                                A.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
                                                ToTensorV2()
                                                ]))

In [23]:
import torch
start = time.time()
loader = torch.utils.data.DataLoader(image_folder)
for batch in loader:
    _, _ = batch
end = time.time()
print('duration:', end-start,'s')

duration: 83.1766836643219 s


Huge speedup by reducing the on-disk image size.